# Setup

Initial module setup.

In [9]:
import numpy.typing as np_types 

from sklearn.svm import LinearSVC
from eeg_auth_models_framework import data, pre_process, features, training, model, processor
from eeg_auth_models_framework.utils import conversion

# Constants

In [10]:
DATASET_SAMPLE_FREQ_HZ = 200
DATA_CHANNEL_NAMES = ['T7','F8','Cz','P4']
FREQUENCIES = [
    pre_process.FrequencyBand(lower=8.0, upper=12.0, label='Alpha'),
    pre_process.FrequencyBand(lower=12.0, upper=35.0, label='Beta'),
    pre_process.FrequencyBand(lower=4.0, upper=8.0, label='Theta'),
    pre_process.FrequencyBand(lower=35.0, upper=None, label='Gamma'),
    pre_process.FrequencyBand(lower=None, upper=None, label='Raw'),
]
WINDOW_SIZE = 1200
WINDOW_OVERLAP = 0.5
K_FOLDS = 10

# Model Builder Configuration

Configure data source, data reading method, data labelling method, and training process. 

In [11]:
class ARSVMBuilder(model.ModelBuilder[LinearSVC]):
    def create_classifier(self) -> LinearSVC:
        return LinearSVC(
            random_state=32,
            dual='auto',
            max_iter=2000
        )
    
    def train_classifier(self, classifier: LinearSVC, x_data: np_types.ArrayLike, y_data: np_types.ArrayLike):
        classifier.fit(x_data, y_data)
        
    def score_classifier(self, classifier: LinearSVC, x_data: np_types.ArrayLike, y_data: np_types.ArrayLike) -> float:
        return classifier.score(x_data, y_data)

# Configuration

In [12]:
downloader = data.AuditoryDataDownloader()
reader = data.AuditoryDataReader()
labeller = training.SubjectDataLabeller()
converter = conversion.MNEDataFrameConverter(
    channels=DATA_CHANNEL_NAMES, 
    sample_frequency=DATASET_SAMPLE_FREQ_HZ
)

# Data Processing

## Pre-Processing Steps

Define pre-processing steps to be used in model.

In [13]:
pre_process_steps = pre_process.PreProcessingPipeline([
    pre_process.EEGBandpassFilterStep(
        FREQUENCIES, 
        converter
    ),
    pre_process.DataWindowStep(WINDOW_SIZE, WINDOW_OVERLAP)
])

## Feature Extraction Steps

Define feature extraction steps to be applied to the pre-processed data.

In [14]:
feature_extraction_steps = features.FeatureExtractPipeline([
    features.ARFeatureExtractor({'lags': 25})
])

## Data Processor

In [15]:
data_processor = processor.DataProcessor(
    pre_process=pre_process_steps,
    feature_extraction=feature_extraction_steps
)

# Training

Execute training of authentication models.

In [16]:
ar_svm_builder = ARSVMBuilder(
    data_downloader=downloader,
    data_reader=reader,
    data_labeller=labeller,
    data_processor=data_processor
)
results = ar_svm_builder.train(K_FOLDS)
print('Average scores and training times:')
for subject, average in results.iter_subject_average_scores():
    training_time = results.training_statistics[subject].training_duration
    print(f'{subject}: {average} ({training_time} seconds)')
print(f'Global average score: {results.global_average_score}')
print(f'Global average training time: {results.global_average_time} seconds')

Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=40114
    Range : 0 ... 40113 =      0.000 ...   200.565 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ..